
# ITMAL Exercise

REVISIONS| |
---------| |
2018-1218| CEF, initial.
2018-0214| CEF, major update.
2018-0220| CEF, added code reference.
2018-0220| CEF, fixed revision table malformatting.
2018-0225| CEF, minor text updates, and made Qc optional.
2019-1008| CEF, updated to ITMAL E19.

## Model capacity and under/overfitting

NOTE: text and code to the exercise it taken from 

* https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html
 

This example demonstrates the problems of underfitting and overfitting and
how we can use linear regression with polynomial features to approximate
nonlinear functions. 

The plot below shows the function that we want to approximate,
which is a part of the cosine function. In addition, the samples from the
real function and the approximations of different models are displayed. The
models have polynomial features of different degrees. We can see that a
linear function (polynomial with degree 1) is not sufficient to fit the
training samples. 

This is called **underfitting**. A polynomial of degree 4
approximates the true function almost perfectly. However, for higher degrees
the model will **overfit** the training data, i.e. it learns the noise of the
training data.

We evaluate quantitatively **overfitting**/**underfitting** by using
cross-validation. We calculate the mean squared error (MSE) on the validation
set, the higher, the less likely the model generalizes correctly from the
training data.

### Qa Explain the polynomial fitting via code review

Review the code below, write a __short__ code review summary, and explain how the polynomial fitting is implemented?

Do not dig into the plotting details, but explain the outcome of the plots.

In [2]:
# NOTE: code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

def true_fun(X):
    return np.cos(1.5 * np.pi * X)

def GenerateData(n_samples = 30):
    X = np.sort(np.random.rand(n_samples))
    y = true_fun(X) + np.random.randn(n_samples) * 0.1
    return X, y

np.random.seed(0)

X, y = GenerateData()
degrees = [1, 4, 15]
    
print("Iterating...degrees=",degrees)
plt.figure(figsize=(14, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures(degree=degrees[i], include_bias=False)
    
    linear_regression = LinearRegression()
    pipeline = Pipeline([
            ("polynomial_features", polynomial_features),
            ("linear_regression", linear_regression)
        ])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y, scoring="neg_mean_squared_error", cv=10)
    
    score_mean = -scores.mean()
    print(f"  degree={degrees[i]:4d}, score_mean={score_mean:4.2f},  {polynomial_features}")   

    X_test = np.linspace(0, 1, 100)
    y_pred = pipeline.predict(X_test[:, np.newaxis])
    
    # Plotting details
    plt.plot(X_test, y_pred          , label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(degrees[i], -scores.mean(), scores.std()))

plt.show()
print('OK')

Iterating...degrees= [1, 4, 15]
  degree=   1, score_mean=0.41,  PolynomialFeatures(degree=1, include_bias=False, interaction_only=False,
                   order='C')
  degree=   4, score_mean=0.04,  PolynomialFeatures(degree=4, include_bias=False, interaction_only=False,
                   order='C')
  degree=  15, score_mean=182815433.48,  PolynomialFeatures(degree=15, include_bias=False, interaction_only=False,
                   order='C')


OK


### Qa Answers) - Code review:

__true_fun:__ This function applies a cosine fuction to the input - this is our true function (the function we want our model to generalize as well as possible)
__GenerateData__: This function enereates data points according to the param n_samples, by default 30 samples. Since the generated X-values are randomized these are first sorted then y (the "true" values for the corresponding X-values) is defines as the sum of the function of true_fun to X and some noise

__Polynomial Feature implementation:__ Using polynomial features a linear regression model can be used to describe non-linear data. It does this by extending the feature set in the the given samples by the defined degree, and then training the regression model on this new set of data with extended features. A 2nd-degree polynomialFeatures model adds the square of all features as new features. Generally a nth-degree model will add all the possible combinations of features up to that nth-degree transforming a n feature array to a array containing a number of features equal to:

$$\frac{n!+d!}{d!n!}$$

__(n being the number of original features and d being the polynomial degree)__

The rest of the cell then runs three iterations (one for each defined degree option): Trainig data is generated and then a pipeline is created which createas a model from of  a LinearRegressor and polynomial features applied to the specified degree. Then a fit-predict is performed on the model, generating the three plots illustrating the fit of each model. They are all showing the same true function and data samples, the polynomial regression models varies:
1st-degree being underfitted, 4th-degree being a good match and 15th-degree being a clear overfit.

### Qb Explain the capacity and under/overfitting concept

Write a textual description of the capacity and under/overfitting concept using the plots in the code above.

What happens when the polynomial degree is low/medium/high with respect to under/overfitting  concepts? Explain in details.

### Qb Answers)

__Model Capacity:__ Model capacity describe how complex a data relationship our model can represent. With a high model capacity for our model, this can describe a complex feature set, but is also far more likely to overfit the data, including any noise or error into the model. If we have a complex data set, then a low capacity model will likely underfit our data and be equally bad at providing any generalization.

__Underfitting:__ Underfitting happens when our model's capacity is too low. Then it will not possibly be able to accurately represent our feature set. An example of such is the 1st plot describing our model run with a polynomial degree of 1. In this case the model is purely a linear regression and as such it cannot properly describe the cosine relation of our true function.

__Overfitting:__ Overfitting happens when our model's capacity is too high. Then it will not be able to generalize ur chosen feature set, resulting a model having learned all training data points including errors and outliers. An example of such is the 3rd plot describing our model run with a polynomial degree of 15. In this case the model is almost descretly fitted to all the training samples and therefore poorly generalizing and describing the cosine relation of our true function.

__Defining a well fitting model:__
Having discussed under- and overfitting, it is clear that it is imporant to be catious about model complexity. When using polynomial regression it is important to choose a good polynomial degree such as shown in the 2nd plot. This model has a degree of 4, and it fits the true function very well. However, it is obviously not always be as easy to define the right degree as in this simple case. A useful tool for choosing such a hyperparameter as polynomial degree is cross-validation.

### [OPTIONAL] Qc Score method

Why is the scoring method called `neg_mean_squared_error` in the code? What happens if you try to set it to `mean_squared_error`?

```python
scores = cross_val_score(pipeline, X[:, np.newaxis], y, scoring="mean_squared_error", cv=10)
```

Why does the Degree 15 model have a MSE of 1.81E8, and hence not the best MSE?

In [ ]:
# TODO: Qc...